# Faceswap Deepfake Trainer Notebook

**Descripción:**

Este notebook permite generar un video de intercambio de rostros (deepfake) usando deep learning por medio del uso del software open-source desarrollado por Faceswap (https://faceswap.dev/), el notebook permite realizar el procesamiento en Colab por medio de las siguientes funcionalidades:

*   Extracción -> Genera un conjunto de dataframes (origen-destino) con base en los 2 videos que sean usados para realizar el deepfake.
*   Entrenamiento Modelo -> Realiza el entrenamiento del modelo, codificando y decodificando las imagenes de acuerdo con un modelo seleccionado.
*   Conversión -> Genera el video deepfake de acuerdo con el modelo entrenado, los dataframes y el perfilamiento.

**Recomendaciones:**

El entrenamiento del modelo puede tardar entre 24 y 48 de acuerdo con la capacidad de procesamiento y varias semanas si no se cuenta uso de GPU, por tal motivo se recomienda ejecutar colab con alguna de las siguientes opciones:

*   Colab Pro -> https://colab.research.google.com/signup
*   VM de GCE de Colab -> A través de GCP Marketplace https://research.google.com/colaboratory/marketplace.html


**Código Fuente:**

https://github.com/deepfakes/faceswap


## Descarga de paquetes e Instalación Faceswap

In [2]:
!git clone https://github.com/christiancgil/faceswap.git 
!pip install -r /content/faceswap/requirements/requirements_nvidia.txt

rm: cannot remove '/content/faceswap': No such file or directory
Cloning into 'faceswap'...
remote: Enumerating objects: 11928, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 11928 (delta 25), reused 71 (delta 24), pack-reused 11854
Receiving objects: 100% (11928/11928), 196.56 MiB | 22.05 MiB/s, done.
Resolving deltas: 100% (8430/8430), done.
Ignoring scikit-learn: markers 'python_version >= "3.9"' don't match your environment
Ignoring pywin32: markers 'sys_platform == "win32"' don't match your environment
Ignoring numpy: markers 'python_version >= "3.8"' don't match your environment
Ignoring pynvx: markers 'sys_platform == "darwin"' don't match your environment
     |████████████████████████████████| 78 kB 3.4 MB/s 
     |████████████████████████████████| 379 kB 20.5 MB/s 
     |████████████████████████████████| 60.9 MB 1.2 MB/s 
     |████████████████████████████████| 194 kB 67.5 MB/s 
     |████████████████████████

## Extracción de rostros y generación archivo de perfilamiento

In [3]:

!git clone https://github.com/christiancgil/faceswap.git
!mkdir '/content/faceswap/src'
!mkdir '/content/faceswap/dst'
!mkdir '/content/faceswap/model'
!mkdir /content/faceswap/converted
!git clone https://github.com/christiancgil/DFLWorkspace.git
!ls -la
# Firts time configuration, Please select the required backend - Seleccionar 3 and Enter
# 1: AMD, 2: CPU, 3: NVIDIA, 4: APPLE SILICON: 3
!python '/content/faceswap/faceswap.py' extract -i '/content/DFLWorkspace/data_src.mp4' -o '/content/faceswap/dst' -al /content/faceswap/model/alignments_src.fsa -D s3fd -A cv2-dnn -M vgg-clear -nm none -rf 0 -min 0 -l 0.4 -sz 512 -een 1 -si 0 -L INFO
!python '/content/faceswap/faceswap.py' extract -i '/content/DFLWorkspace/data_dst.mp4' -o '/content/faceswap/scr' -al /content/faceswap/model/alignments_dst.fsa -D s3fd -A cv2-dnn -M vgg-clear -nm none -rf 0 -min 0 -l 0.4 -sz 512 -een 1 -si 0 -L INFO

fatal: destination path 'faceswap' already exists and is not an empty directory.
Cloning into 'DFLWorkspace'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 42 (delta 1), reused 7 (delta 1), pack-reused 35
Unpacking objects: 100% (42/42), done.
total 24
drwxr-xr-x  1 root root 4096 Oct 14 01:13 .
drwxr-xr-x  1 root root 4096 Oct 14 01:08 ..
drwxr-xr-x  4 root root 4096 Feb 18  2022 .config
drwxr-xr-x  6 root root 4096 Oct 14 01:14 DFLWorkspace
drwxr-xr-x 19 root root 4096 Oct 14 01:13 faceswap
drwxr-xr-x  2 root root 4096 Feb 18  2022 sample_data
First time configuration. Please select the required backend
1: AMD, 2: CPU, 3: NVIDIA, 4: APPLE SILICON: 3
Faceswap config written to: /content/faceswap/config/.faceswap
Setting Faceswap backend to NVIDIA
10/14/2022 01:14:37 INFO     Log level set to: INFO
10/14/2022 01:14:40 INFO     Loading Detect from S3Fd plugin...
10/14/2022 01:14:40 INFO  

## Limpieza de las imagenes generadas y eliminación de imagenes duplicadas

In [4]:
!python '/content/faceswap/tools.py' alignments -j multi-faces -o move -a /content/faceswap/model/alignments_src.fsa -fc /content/faceswap/src -een 1 -sz 512 -m 0 -L INFO
!python '/content/faceswap/tools.py' alignments -j multi-faces -o move -a /content/faceswap/model/alignments_dst.fsa -fc /content/faceswap/dst -een 1 -sz 512 -m 0 -L INFO
!rm -r /content/faceswap/src/multi*
!rm -r /content/faceswap/dst/multi*

Setting Faceswap backend to NVIDIA
10/14/2022 01:23:33 INFO     Log level set to: INFO
10/14/2022 01:23:36 INFO     [ALIGNMENT DATA]
10/14/2022 01:23:36 INFO     Reading alignments from: '/content/faceswap/model/alignments_src.fsa'
10/14/2022 01:23:36 INFO     [FACES DATA]
10/14/2022 01:23:36 INFO     Loading file list from /content/faceswap/src
Reading Face Data: 100% 655/655 [00:01<00:00, 594.13it/s]
10/14/2022 01:23:37 INFO     [CHECK FACES]
10/14/2022 01:23:37 INFO     Moving 2 faces(s) to '/content/faceswap/src/multiple_faces_in_frame_20221014_012337'
Setting Faceswap backend to NVIDIA
10/14/2022 01:23:38 INFO     Log level set to: INFO
10/14/2022 01:23:41 INFO     [ALIGNMENT DATA]
10/14/2022 01:23:41 INFO     Reading alignments from: '/content/faceswap/model/alignments_dst.fsa'
10/14/2022 01:23:41 INFO     [FACES DATA]
10/14/2022 01:23:41 INFO     Loading file list from /content/faceswap/dst
Reading Face Data: 100% 1450/1450 [00:01<00:00, 820.53it/s]
10/14/2022 01:23:43 INFO     

In [6]:
#Donwload Trained Model 20 horas y 240.000 repeticiones
#!pip install --upgrade --no-cache-dir gdown
#!rm -r /content/DeepFake-DemoFiles
#!gdown --folder https://drive.google.com/drive/folders/14cc8QjwxDHAkHPL_NVvWt7MEu7SEBjAM
#!ls -la /content/DeepFake-DemoFiles
#!mkdir /content/faceswap/model
#!mv /content/DeepFake-DemoFiles/* /content/faceswap/model
#!ls -la /content/faceswap/model

## Entrenamiento del Modelo con 300.000 repeticiones

In [7]:
#Entrenamiento Modelo
!mkdir '/content/faceswap/timelapse'
!python '/content/faceswap/faceswap.py' train -A '/content/faceswap/src' -B '/content/faceswap/dst' -m '/content/faceswap/model' -t original -it 300000 -s 250 -tia /content/faceswap/src -tib /content/faceswap/dst -to /content/faceswap/timelapse -L INFO

Setting Faceswap backend to NVIDIA
10/14/2022 01:23:44 INFO     Log level set to: INFO
10/14/2022 01:23:47 INFO     Model A Directory: '/content/faceswap/src' (653 images)
10/14/2022 01:23:47 INFO     Model B Directory: '/content/faceswap/dst' (1448 images)
10/14/2022 01:23:47 INFO     Training data directory: /content/faceswap/model
10/14/2022 01:23:47 INFO     ===================================================
10/14/2022 01:23:47 INFO       Starting
10/14/2022 01:23:47 INFO       Press 'ENTER' to save and quit
10/14/2022 01:23:47 INFO       Press 'S' to save model weights immediately
10/14/2022 01:23:47 INFO     ===================================================
10/14/2022 01:23:47 INFO     Loading data, this may take a while...
10/14/2022 01:23:47 INFO     Loading Model from Original plugin...
10/14/2022 01:23:48 INFO     No existing state file found. Generating.
10/14/2022 01:23:48 INFO     Storing Mixed Precision compatible layers. Please ignore any following warnings about usin

## Descarga de Modelo Entrenado

In [10]:
!ls -lst '/content/faceswap/model'
!zip -r /content/model.zip /content/faceswap/model/original.h5
from google.colab import files
files.download('/content/model.zip')

total 974040
     4 -rw-r--r-- 1 root root      2758 Oct 14 20:15 original_state.json
320612 -rw-r--r-- 1 root root 328306368 Oct 14 20:15 original.h5
320612 -rw-r--r-- 1 root root 328306368 Oct 14 20:14 original.h5.bk.dw
     4 -rw-r--r-- 1 root root      2757 Oct 14 19:57 original_state.json.bk
320612 -rw-r--r-- 1 root root 328306368 Oct 14 19:57 original.h5.bk
     4 drwxr-xr-x 3 root root      4096 Oct 14 01:23 original_logs
  8156 -rw-r--r-- 1 root root   8348540 Oct 14 01:23 alignments_dst.fsa
  4036 -rw-r--r-- 1 root root   4129736 Oct 14 01:18 alignments_src.fsa


## Creación de video deepfake con intercambio rostros

In [13]:
# Crear video con el intercambio de caras
!mkdir /content/faceswap/converted
!python '/content/faceswap/faceswap.py' convert -i '/content/DFLWorkspace/data_src.mp4' -o '/content/faceswap/converted' -m '/content/faceswap/model' -al /content/faceswap/model/alignments_src.fsa -c avg-color -M extended -w ffmpeg -osc 100 -l 0.4 -j 0 -L INFO


Setting Faceswap backend to NVIDIA
10/14/2022 20:18:21 INFO     Log level set to: INFO
10/14/2022 20:18:26 INFO     Reading alignments from: '/content/faceswap/model/alignments_src.fsa'
10/14/2022 20:18:26 INFO     Loading Writer from Ffmpeg plugin...
10/14/2022 20:18:26 INFO     Outputting to: '/content/faceswap/converted/data_src_converted.mp4'
10/14/2022 20:18:26 INFO     Loading Model from Original plugin...
10/14/2022 20:18:27 INFO     Using configuration saved in state file
10/14/2022 20:18:32 INFO     Loaded model from disk: '/content/faceswap/model/original.h5'
10/14/2022 20:18:32 INFO     Loading Mask from Mask_Blend plugin...
10/14/2022 20:18:32 INFO     Loading Color from Avg_Color plugin...
10/14/2022 20:18:32 INFO     Loading Scaling from Sharpen plugin...
10/14/2022 20:18:42 WARNING  No audio stream could be found in the source video '/content/DFLWorkspace/data_src.mp4'. Muxing audio will be disabled.
Converting: 100% 655/656 [03:14<00:00,  3.36it/s]
10/14/2022 20:21:44 I

## Descarga video deepfake

In [14]:
# Donwload deepfake video
from google.colab import files
files.download('/content/faceswap/converted/data_src_converted.mp4')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>